In [2]:
import cv2
import numpy as np
import time
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max

In [3]:
def contrast_strech(image):
    a = 0
    b = 255
    c = image.min()
    d_max = image.max()
    image_new = image
    image_new[:,:,:] = (image[:,:,:] - c)* ((b-a)/(d_max - c)) + a #(I-c)(b-a/max - min) + a
    return image_new

In [4]:
def convolve_max_filtered(image,N):
    h,w = image.shape
    output = np.zeros((h,w),dtype = np.uint8)
    #filter_new = np.array([[1]*N]*N  ,dtype ="int")  ## N*N filter with 1 in all places
    #need to pad the image according to filter size
    p = int((N-1)/2) #paading of zeros required
    padded_picture = cv2.copyMakeBorder(image.copy(),p,p,p,p,cv2.BORDER_CONSTANT,value = 0)
    for y in np.arange(p,h+p):
        for x in np.arange(p,w+p):
                roi = padded_picture[y-p:y+p+1, x-p:x+p+1]
                max_pixel = np.amax((roi)) #directly finding the max value from surrounding pixels and replacing it
                output[y-p,x-p]= max_pixel
    output = (output*255).astype("uint8")
    return output

In [5]:
def convolve_min_filtered(image,N):
    h,w = image.shape
    output = np.full((h,w),255,dtype = np.uint8)
    #filter_new = np.array([[1]*N]*N  ,dtype ="int")  ## N*N filter with 1 in all places
    #need to pad the image according to filter size
    p = int((N-1)/2) #paading of zeros required
    padded_picture = cv2.copyMakeBorder(image.copy(),p,p,p,p,cv2.BORDER_CONSTANT,value = 255)
    
    for y in np.arange(p,h+p):
        for x in np.arange(p,w+p):
                roi = padded_picture[y-p:y+p+1, x-p:x+p+1]
                min_pixel = np.amin((roi)) #directly finding the max value from surrounding pixels and replacing it
                output[y-p,x-p]= min_pixel
           
    output = (output*255).astype("uint8")
    return output

In [6]:
def remove_background(image,back_img):
    h,w = image.shape
    output = np.zeros((h,w),dtype = "uint8")
    output = np.subtract(image,back_img)
    return output

In [7]:
def run_algo(image,filter_size, M):
    if M == 0:
        max_filter =  convolve_max_filtered(image,filter_size)
        #min_filter =  convolve_min_filtered(max_filter,filter_size)
        final_image = remove_background(image, max_filter)
    else:
        min_filter = convolve_min_filtered(image,filter_size)
        #max_filter = convolve_max_filtered(min_filter,filter_size)
        final_image = remove_background(image, min_filter)
    return final_image

In [8]:
def apply_watershed(img):
    img_array = img.copy()
    distance = ndi.distance_transform_edt(img_array)
    markers = ndi.label(peak_local_max(distance, indices = False, labels = img_array))[0]
    ws_labels = watershed(-distance, markers, mask = img_array)
    print("[INFO] {} unique segments found".format(len(np.unique(ws_labels)) - 1))
    return ws_labels

In [9]:
def image_sharpening(image):
    sigma = 10
    a = 10
    blur_image = cv2.GaussianBlur(image,(0,0),sigma)  #L = I*G
    #H = image - blur_image #H = I - L
    H = cv2.subtract(image,blur_image)
    final_image = H*a + image #o = I+H*a
    return final_image

In [106]:
def make_boxes(image,labels):       
    conts=[]    
    for label in np.unique(labels):
        # if the label is zero, we are examining the 'background'
        # so simply ignore it
        if label == 0:
            continue
        # otherwise, allocate memory for the label region and draw
        # it on the mask
        mask = np.zeros(gray.shape, dtype="uint8")
        mask[labels == label] = 255
        # detect contours in the mask and grab the largest one
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)[-2]
        c = max(cnts, key=cv2.contourArea)
        rect = cv2.minAreaRect(c)
        box = cv2.boxPoints(rect)
        box = np.int0(box)  
        if cv2.contourArea(c) > 50: 
            #cv2.drawContours(image,c,-1,(0,255,0))
            cv2.drawContours(image,[box],-1,(128,128,128),thickness = 1)      
    return image

In [107]:
cap = cv2.VideoCapture("C:/Users/Deepansh/Desktop/COMP 9517/GROUP PROJECT/images/Fluo-N2DL-HeLa/Sequence 4/t%03d.tif")
filter_size = 3
M = 0
while(1):
    ret, frame = cap.read()
    if (ret) :  
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('original image',gray)
        #applying min_max filtering
        #frame = run_algo(gray, filter_size,M)
        kernel_erosion = np.ones((3,3),np.uint8)
        erosion = cv2.erode(gray,kernel_erosion,iterations = 1)
        kernel_dilation = np.ones((5,5), np.uint8)
        img_dilation = cv2.dilate(erosion, kernel_dilation, iterations=1)
        th = cv2.adaptiveThreshold(img_dilation,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        frame = remove_background(th,img_dilation)
        blur = cv2.GaussianBlur(frame,(5,5),0)
        ret,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        frame = cv2.bitwise_not(th)
        cv2.imshow('thresholded image',frame)
        label = apply_watershed(frame)
        frame_boxes = make_boxes(frame,label)
        cv2.imshow('BOXES',frame_boxes)
        time.sleep(0.5)
    else:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    # 'q' to quit playback
    if (cv2.waitKey(1) & 0xFF == ord('q')):
        break
        
cap.release()
cv2.destroyAllWindows()

D:\ANACONDA\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


[INFO] 169 unique segments found


D:\ANACONDA\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


[INFO] 185 unique segments found


D:\ANACONDA\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


...

[INFO] 325 unique segments found


D:\ANACONDA\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


[INFO] 340 unique segments found


D:\ANACONDA\lib\site-packages\skimage\morphology\_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


[INFO] 333 unique segments found
